**Do datasets work?**
* I think I don't need the `isinstance` dataset with `.copy()` if I can just use the `apply_ufunc`.

**Other stuff**
* Various NaN policies (return NaN, interpolate, skip)

In [ ]:
%matplotlib inline
import numpy as np
import xarray as xr
import numpy.polynomial.polynomial as poly
import cftime

In [ ]:
from esmtools.utils import convert_time

def _compute_slope(x, y):
    """
    Private function to compute slopes at each grid cell using
    polyfit. 
    """
    return np.polyfit(x, y, 1)[0] # return only the slope

def compute_slope(x, y, dim='time'):
    """
    Computes linear slope (m) at each grid cell.
    
    Args:
      da: xarray DataArray to compute slopes for
      
    Returns:
      xarray DataArray with slopes computed at each grid cell.
    """
    x = convert_time(x, dim)
    
    slopes = xr.apply_ufunc(_compute_slope,
                            x,
                            y,
                            vectorize=True,
                            dask='parallelized', 
                            input_core_dims=[[dim], [dim]],
                            output_dtypes=[float],
                            )
    return slopes

In [ ]:
DAYS_PER_MONTH = {
    'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    '365_day': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'standard': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'all_leap': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    '366_day': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30],
    'julian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
}

In [ ]:
{k: np.sum(v) for (k, v) in DAYS_PER_MONTH.items()}

In [ ]:
np.sum(DAYS_PER_MONTH['noleap'])

In [ ]:
ds = xr.open_dataset('/Users/ribr5703/Desktop/dev/climpred-data/CESM-DP-LE.SST.eastern_pacific.nc')
ds = ds.SST.isel(lead=0).rename({'init': 'time'})

In this case, we give integers for time, so it just computes the delta x equally and returns the slope in units of per year.

In [ ]:
ds.time

In [ ]:
A = compute_slope(ds.time, ds)

`numpy` datetime

In [ ]:
ds = xr.open_dataset('/Users/ribr5703/Desktop/dev/climpred-data/CESM-DP-LE.SST.eastern_pacific.nc')
ds = ds.SST.isel(lead=0).rename({'init': 'time'})
ds['time'] = np.arange('1900-12-31', '1964-12-31', dtype='datetime64[Y]')

In [ ]:
B = compute_slope(ds.time, ds)

`cftime` datetime

In [ ]:
ds = xr.open_dataset('/Users/ribr5703/Desktop/dev/climpred-data/CESM-DP-LE.SST.eastern_pacific.nc')
ds = ds.SST.isel(lead=0).rename({'init': 'time'})
ds['time'] = xr.cftime_range('1900', '1964', freq='Y')

In [ ]:
C = compute_slope(ds.time, ds)

In [ ]:
assert (isinstance(ds.time, xr.CFTimeIndex) or np.issubdtype(ds.time, np.datetime64)), f'{type(test)} is not a datetime index'